In [1]:
import torch
import torch.nn as nn
import numpy
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [2]:
rate=0.001
epoch=20
batch_size = 32

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

print(device)

cuda


In [4]:
mnist_train = dsets.MNIST( root = 'MNIST_data/',
                          train= True,
                          transform = transforms.ToTensor(),
                          download=True)
 
mnist_test = dsets.MNIST( root = 'MNIST_data/',
                          train= False,
                          transform = transforms.ToTensor(),
                          download=True)

In [5]:
data = torch.utils.data.DataLoader(mnist_train, batch_size, shuffle=True)

In [6]:
from torch.nn.init import kaiming_normal_

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        kaiming_normal_(self.layer1[0].weight, mode='fan_in', nonlinearity='relu')
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        kaiming_normal_(self.layer2[0].weight, mode='fan_in', nonlinearity='relu')
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        kaiming_normal_(self.layer3[0].weight, mode='fan_in', nonlinearity='relu')
        
        self.fc1 = nn.Linear(3 * 3 * 128, 625)
        self.fc2 = nn.Linear(625, 10, bias=True)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        
        return out


In [7]:
model = CNN().to(device)
cr = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = rate)

In [8]:
total_batch = len(data)

for e in range(epoch):
    avg_cost = 0
    for x, y in data:
        optimizer.zero_grad()
        x = x.to(device)
        y = y.to(device)

        hy = model(x)
        cost = cr(hy, y)

        cost.backward()
        optimizer.step()
        avg_cost += cost / total_batch

    if (e + 1) % 1 == 0:
        print(f"Epoch {e + 1}/{epoch}, Average Cost: {avg_cost.item()}")

print("학습 완료")


Epoch 1/20, Average Cost: 0.12625433504581451
Epoch 2/20, Average Cost: 0.055289581418037415
Epoch 3/20, Average Cost: 0.04108230024576187
Epoch 4/20, Average Cost: 0.03508155047893524
Epoch 5/20, Average Cost: 0.03073696978390217
Epoch 6/20, Average Cost: 0.022700216621160507
Epoch 7/20, Average Cost: 0.02255374938249588
Epoch 8/20, Average Cost: 0.018457697704434395
Epoch 9/20, Average Cost: 0.019076131284236908
Epoch 10/20, Average Cost: 0.015502030961215496
Epoch 11/20, Average Cost: 0.015695104375481606
Epoch 12/20, Average Cost: 0.016246812418103218
Epoch 13/20, Average Cost: 0.010698878206312656
Epoch 14/20, Average Cost: 0.01602201722562313
Epoch 15/20, Average Cost: 0.011158851906657219
Epoch 16/20, Average Cost: 0.012186906300485134
Epoch 17/20, Average Cost: 0.013085709884762764
Epoch 18/20, Average Cost: 0.011621943674981594
Epoch 19/20, Average Cost: 0.012565265409648418
Epoch 20/20, Average Cost: 0.010342895984649658
학습 완료


In [9]:
with torch.no_grad():
    x_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    y_test = mnist_test.test_labels.to(device)
    pr = model(x_test)
    co_pr = torch.argmax(pr, 1) == y_test
    accuracy = co_pr.float().mean()
    print("Accuracy:",accuracy.item()*100)

print("테스트 완료")

C:\Users\kuroc\anaconda3\Lib\site-packages\torchvision\datasets\mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\kuroc\anaconda3\Lib\site-packages\torchvision\datasets\mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 93.50999593734741
테스트 완료
